In [ ]:
import pandas as pd
df_sample = pd.read_csv("/Users/htg/Desktop/Sydney_University/冒犯性检测/dataset/data_emnlp.csv", sep=",")

In [ ]:
#Chatgpt o3-mini

import pandas as pd
from openai import OpenAI

# 读取数据集
texts = df_sample.iloc[:, 0]

api_key ="Your own key"
client = OpenAI(api_key=api_key)

results = []

for idx, line in enumerate(texts):
    prompt = (
    "你是一位服务中文社交媒体的评论审核专家。\n"
    "请判断下面的文本是否具有冒犯性，文本可能包含谐音，表情等内容\n"
    "这里的“冒犯性”主要指包含人身攻击、侮辱、歧视、仇恨言论或极端粗俗的内容。\n" 
    "如果文本具有冒犯性，请仅回复数字 1；如果不具有冒犯性，请仅回复数字 0。\n"
    f"文本如下：\n{line}\n"
    )   

    stream = client.chat.completions.create(
    model="o3-mini-2025-01-31",
    messages=[{"role": "user", "content": prompt}],
    max_completion_tokens=5000,
    stop=None,  
    stream=True
    )
    result = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            result += chunk.choices[0].delta.content
    print(f"第 {idx+1} 条数据判断结果：{result.strip()}")
    results.append(result.strip())

# 保存结果
result_df = pd.DataFrame({
    "Content": texts,
    "Result": results
})

result_df.to_csv("./base_result/chatgpt_o3-mini_results.csv", sep="\t", index=False)

In [ ]:
#Chatgpt 4o
import pandas as pd
from openai import OpenAI

# 读取数据集
texts = df_sample.iloc[:, 0]

api_key ="Your own key"
client = OpenAI(api_key=api_key)

results = []

for idx, line in enumerate(texts):
    prompt = (
    "你是一位服务中文社交媒体的评论审核专家。\n"
    "请判断下面的文本是否具有冒犯性，文本可能包含谐音，表情等内容\n"
    "这里的“冒犯性”主要指包含人身攻击、侮辱、歧视、仇恨言论或极端粗俗的内容。\n" 
    "如果文本具有冒犯性，请仅回复数字 1；如果不具有冒犯性，请仅回复数字 0。\n"
    f"文本如下：\n{line}\n"
    )   

    stream = client.chat.completions.create(
    model="gpt-4o-2024-11-20",
    messages=[{"role": "user", "content": prompt}],
    max_completion_tokens=5000,
    temperature=0.1,
    stop=None,  
    stream=True
    )
    result = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            result += chunk.choices[0].delta.content
    print(f"第 {idx+1} 条数据判断结果：{result.strip()}")
    results.append(result.strip())

# 保存结果
result_df = pd.DataFrame({
    "Content": texts,
    "Result": results
})

result_df.to_csv("./base_result/chatgpt_4o_results.csv", sep="\t", index=False)

In [ ]:
#Chatgpt 4o COT

import pandas as pd
from openai import OpenAI
import re

# 读取数据集
texts = df_sample.iloc[:, 0]

api_key ="Your own key"
client = OpenAI(api_key=api_key)

results = []
COT = []

for idx, line in enumerate(texts):
    prompt = (
        f"""
                            你是一个中文冒犯性语言检测专家
                            请判断下面的文本是否具有冒犯性，文本可能包含谐音，表情等内容。
                            这里的“冒犯性”主要指包含人身攻击、侮辱、歧视、仇恨言论或极端粗俗的内容。
                            先一步步分析理由，然后在最后一行给出结论：
                            - 如果冒犯，只输出 1
                            - 如果不冒犯，只输出 0

                            输出格式:
                            思考过程: <逐步推理>
                            结论: <0或1>

                            文本如下：
                            {line} 
                            """
    )   

    stream = client.chat.completions.create(
    model="gpt-4o-2024-11-20",
    messages=[{"role": "user", "content": prompt}],
    max_completion_tokens=3000,
    temperature=0.1,
    stop=None,  
    stream=True
    )
    result = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            result += chunk.choices[0].delta.content
    match_think = re.search(r"思考过程\s*[:：]\s*([\s\S]*?)结论", result)
    think_text = match_think.group(1).strip() if match_think else ""      
    match = re.search(r"结论\s*[:：]\s*([01])", result)
    label = match.group(1) if match else ""
    results.append(label)
    COT.append(think_text)
    print(f"第 {idx + 1} 条数据COT：{think_text}")
    print(f"第 {idx + 1} 条数据判断结果：{label}")

# 保存结果
result_df = pd.DataFrame({
    "Content": texts,
    "COT": COT,
    "Result": results
})

result_df.to_excel("./COT_result/chatgpt_4o_COT_results.xlsx", index=False)